# Otimização do Modelo WDO
Otimização dos parâmetros e validação cruzada do modelo.

In [17]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_ROOT = Path(os.path.dirname(os.getcwd()))
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

plt.style.use('default')
pd.set_option('display.max_columns', None)

## 1. Grid Search de Parâmetros

In [18]:
# Carrega dados
from src.data.loaders.market_data_loader import MarketDataLoader
from src.ml.features.feature_engineering import FeatureEngineering
from src.agents.market_agent import RiskManagementAgent, MarketRegimeAgent
from src.agents.agent_coordinator import AgentCoordinator
from src.backtest.monte_carlo import MonteCarloSimulator

# Configuração dos parâmetros para teste
param_grid = {
    'risk_management': {
        'max_drawdown': [-0.03, -0.04, -0.05, -0.06],
        'max_position_size': [2, 3, 4, 5]
    },
    'market_regime': {
        'window_size': [10, 15, 20, 25],
        'volatility_threshold': [0.1, 0.15, 0.2]
    }
}

# Carrega dados e modelo otimizado
db_path = PROJECT_ROOT / 'src' / 'data' / 'database' / 'candles.db'
loader = MarketDataLoader(str(db_path))
data = loader.load_data()

engineer = FeatureEngineering()
features = engineer.create_technical_features(data)
features = engineer.create_temporal_features(features)

# Carrega modelo com parâmetros otimizados
risk_agent = RiskManagementAgent(
    max_drawdown=-0.002793,
    max_position_size=2
)

regime_agent = MarketRegimeAgent(
    window_size=25
)

coordinator = AgentCoordinator([risk_agent, regime_agent])

In [19]:
# Função para avaliar uma combinação de parâmetros
def evaluate_parameters(features, risk_params, regime_params):
    risk_agent = RiskManagementAgent(**risk_params)
    regime_agent = MarketRegimeAgent(**regime_params)
    coordinator = AgentCoordinator([risk_agent, regime_agent])
    
    tscv = TimeSeriesSplit(n_splits=5)
    metrics_list = []
    
    for train_idx, test_idx in tscv.split(features):
        train_data = features.iloc[train_idx]
        test_data = features.iloc[test_idx]
        
        simulator = MonteCarloSimulator(n_simulations=100)
        returns = test_data['returns'].dropna()
        simulation = simulator.simulate_returns(
            historical_returns=returns,
            initial_capital=100000
        )
        
        # Cálculo manual das métricas
        equity_curve = simulation.equity_curves.mean(axis=0)
        returns = np.diff(equity_curve) / equity_curve[:-1]
        
        sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
        max_dd = (equity_curve - np.maximum.accumulate(equity_curve)).min() / equity_curve.max()
        
        metrics = {
            'sharpe_ratio': sharpe,
            'max_drawdown': max_dd,
            'profit_factor': len(returns[returns > 0]) / len(returns[returns < 0]),
            'win_rate': len(returns[returns > 0]) / len(returns)
        }
        metrics_list.append(metrics)
    
    avg_metrics = pd.DataFrame(metrics_list).mean()
    return avg_metrics

# Grid search
param_grid = {
    'risk_management': {
        'max_drawdown': [-0.03, -0.04, -0.05, -0.06],
        'max_position_size': [2, 3, 4, 5]
    },
    'market_regime': {
        'window_size': [10, 15, 20, 25]
    }
}

results = []
for max_dd in param_grid['risk_management']['max_drawdown']:
    for pos_size in param_grid['risk_management']['max_position_size']:
        for window in param_grid['market_regime']['window_size']:
            risk_params = {
                'max_drawdown': max_dd,
                'max_position_size': pos_size
            }
            regime_params = {
                'window_size': window
            }
            
            metrics = evaluate_parameters(features, risk_params, regime_params)
            results.append({
                **risk_params,
                **regime_params,
                **metrics.to_dict()
            })

results_df = pd.DataFrame(results)
print('\nMelhores parâmetros por métrica:')
print('\nMelhor Sharpe Ratio:')
print(results_df.loc[results_df['sharpe_ratio'].idxmax()])

print('\nMenor Max Drawdown:')
print(results_df.loc[results_df['max_drawdown'].idxmin()])


Melhores parâmetros por métrica:

Melhor Sharpe Ratio:
max_drawdown         -0.003133
max_position_size     2.000000
window_size          15.000000
sharpe_ratio          0.978399
profit_factor         1.103564
win_rate              0.523834
Name: 33, dtype: float64

Menor Max Drawdown:
max_drawdown         -0.006609
max_position_size     2.000000
window_size          10.000000
sharpe_ratio          0.655240
profit_factor         1.066382
win_rate              0.515354
Name: 48, dtype: float64


## 2. Validação dos Melhores Parâmetros

In [20]:
# Usando os melhores parâmetros encontrados
best_params_idx = results_df['sharpe_ratio'].idxmax()
best_params = results_df.loc[best_params_idx]

risk_agent = RiskManagementAgent(
   max_drawdown=best_params['max_drawdown'],
   max_position_size=best_params['max_position_size']
)

regime_agent = MarketRegimeAgent(
   window_size=best_params['window_size']
)

coordinator = AgentCoordinator([risk_agent, regime_agent])

# Teste final com dados mais recentes
test_size = int(len(features) * 0.2)  # 20% mais recentes
train_data = features[:-test_size]
test_data = features[-test_size:]

simulator = MonteCarloSimulator(n_simulations=1000)
returns = test_data['returns'].dropna()
simulation = simulator.simulate_returns(
   historical_returns=returns,
   initial_capital=100000
)

# Cálculo das métricas
equity_curve = simulation.equity_curves.mean(axis=0)
returns = np.diff(equity_curve) / equity_curve[:-1]

sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
max_dd = (equity_curve - np.maximum.accumulate(equity_curve)).min() / equity_curve.max()
win_rate = len(returns[returns > 0]) / len(returns)
profit_factor = len(returns[returns > 0]) / len(returns[returns < 0])

print('\nResultados Finais com Parâmetros Otimizados:')
print(f'Sharpe Ratio: {sharpe:.2f}')
print(f'Max Drawdown: {max_dd:.2%}')
print(f'Win Rate: {win_rate:.2%}')
print(f'Profit Factor: {profit_factor:.2f}')


Resultados Finais com Parâmetros Otimizados:
Sharpe Ratio: 4.39
Max Drawdown: -0.01%
Win Rate: 60.73%
Profit Factor: 1.55
